In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
os.chdir(module_path)

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from tqdm import tqdm

In [30]:
train_df = pd.read_csv('data/train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
test_df = pd.read_csv('data/sample_submission.csv', low_memory=False)
properties = pd.read_csv('data/properties_2016.csv', low_memory=False)
# field is named differently in submission
test_df['parcelid'] = test_df['ParcelId']

In [21]:
from interface import *
from config import *
config_dict = getattr(config, 'config_catboost')
Model = config_dict['Model']
train_df, _, _ = load_full_data('config_catboost')

/home/peter/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


name: config_catboost
Model: <class 'models.CatBoost.CatBoost'>
before_fill: 1
original: 38
generated: 0
clean_na: False
training_params: {'model_params': {'iterations': 200, 'learning_rate': 0.03, 'depth': 6, 'l2_leaf_reg': 3, 'loss_function': 'MAE', 'eval_metric': 'MAE', 'random_seed': 42}, 'FOLDS': 5, 'outliers_lw_pct': 0, 'outliers_up_pct': 100}
tuning_params: {'parameter_space': {'model_params': {'iterations': <hyperopt.pyll.base.Apply object at 0x7f323ddecc18>, 'learning_rate': <hyperopt.pyll.base.Apply object at 0x7f323ddecdd8>, 'depth': <hyperopt.pyll.base.Apply object at 0x7f323ddecef0>, 'l2_leaf_reg': <hyperopt.pyll.base.Apply object at 0x7f323ddf61d0>, 'loss_function': 'MAE', 'eval_metric': 'MAE', 'random_seed': 42}, 'FOLDS': 3, 'outliers_up_pct': <hyperopt.pyll.base.Apply object at 0x7f323ddf6390>, 'outliers_lw_pct': <hyperopt.pyll.base.Apply object at 0x7f323ddf65c0>}, 'max_evals': 100}
Feature engineering
Using prop with nan
(2985217, 39)
Load training data...


In [31]:
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)
        
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

KeyError: 'airconditioningtypeid'

In [ ]:
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

In [ ]:
train_df = add_date_features(train_df)
train_df = train_df.merge(properties, how='left', on='parcelid')
test_df = test_df.merge(properties, how='left', on='parcelid')
print("Train: ", train_df.shape)
print("Test: ", test_df.shape)

In [ ]:
missing_perc_thresh = 0.98
exclude_missing = []
num_rows = train_df.shape[0]
for c in train_df.columns:
    num_missing = train_df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print("We exclude: %s" % exclude_missing)
print(len(exclude_missing))

In [ ]:
# exclude where we only have one unique value :D
exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)
print("We exclude: %s" % exclude_unique)
print(len(exclude_unique))

In [ ]:
exclude_other = ['parcelid', 'logerror']  # for indexing/training only
# do not know what this is LARS, 'SHCG' 'COR2YY' 'LNR2RPD-R3' ?!?
exclude_other.append('propertyzoningdesc')
train_features = []
for c in train_df.columns:
    if c not in exclude_missing \
       and c not in exclude_other and c not in exclude_unique:
        train_features.append(c)
print("We use these for training: %s" % train_features)
print(len(train_features))

In [ ]:
# some out of range int is a good choice
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

In [ ]:
X_train = train_df[train_features]
y_train = train_df.logerror
print(X_train.shape, y_train.shape)

In [ ]:
import features.utils

In [ ]:
train_q1_q3, train_q4 = features.utils.split_by_date(train_df)

In [ ]:
X_train_q1_q3, y_train_q1_q3 = features.utils.get_features_target(train_q1_q3)
X_train_q4, y_train_q4 = features.utils.get_features_target(train_q4)

In [ ]:
X_train_q1_q3 = X_train_q1_q3[train_features]
X_train_q4 = X_train_q4[train_features]
X_train_q1_q3.drop(["transactiondate"], inplace=True, axis=1)
X_train_q4.drop(["transactiondate"], inplace=True, axis=1)

In [ ]:
def tostring(name):
    X_train_q1_q3[name] = X_train_q1_q3[name].apply(str)
    X_train_q4[name] = X_train_q4[name].apply(str)

In [ ]:
tostring('rawcensustractandblock')
tostring('censustractandblock')

In [18]:
X_train_q1_q3

,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,...,threequarterbathnbr,unitcnt,yardbuildingsqft17,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock
0,1,1,1,1.0,2.0,3.0,4.0,2.0,-999.0,1684.0,...,-999.0,1.0,-999.0,1959.0,-999.0,122754.0,360170.0,237416.0,6735.88,60371066461001.0
1,1,1,1,-999.0,3.5,4.0,-999.0,3.5,-999.0,2263.0,...,1.0,-999.0,-999.0,2014.0,-999.0,346458.0,585529.0,239071.0,10153.02,-999.0
2,1,1,1,1.0,3.0,2.0,4.0,3.0,-999.0,2217.0,...,-999.0,1.0,-999.0,1940.0,-999.0,61994.0,119906.0,57912.0,11484.48,60374638003004.0
3,1,2,1,1.0,2.0,2.0,4.0,2.0,-999.0,839.0,...,-999.0,1.0,-999.0,1987.0,-999.0,171518.0,244880.0,73362.0,3048.74,60372963002002.0
4,1,2,1,-999.0,2.5,4.0,-999.0,2.5,-999.0,2283.0,...,1.0,-999.0,-999.0,1981.0,2.0,169574.0,434551.0,264977.0,5488.96,60590423381006.0
5,1,2,1,1.0,4.0,4.0,1.0,4.0,-999.0,3067.0,...,-999.0,1.0,-999.0,1982.0,-999.0,880650.0,2447951.0,1567301.0,27126.57,60376210044006.0
6,1,2,1,-999.0,1.0,2.0,7.0,1.0,-999.0,1297.0,...,-999.0,1.0,-999.0,1939.0,-999.0,64549.0,111521.0,46972.0,2304.97,60375416053007.0
7,1,2,1,-999.0,2.5,3.0,-999.0,2.5,853.0,1763.0,...,1.0,-999.0,-999.0,1994.0,2.0,107000.0,306000.0,199000.0,3745.50,61110031002073.0
8,1,2,1,-999.0,1.0,2.0,-999.0,1.0,-999.0,796.0,...,-999.0,-999.0,-999.0,1984.0,-999.0,66834.0,210064.0,143230.0,2172.88,60590423151018.0
9,1,3,1,-999.0,2.0,2.0,-999.0,2.0,-999.0,1260.0,...,-999.0,-999.0,-999.0,1977.0,1.0,109977.0,190960.0,80983.0,1940.26,60590626483015.0


In [ ]:
train_features = train_features[1:]

In [11]:
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(X_train_q1_q3[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)
        
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

NameError: name 'X_train_q1_q3' is not defined

In [ ]:
test_df['transactiondate'] = pd.Timestamp('2016-12-01')  # Dummy
test_df = add_date_features(test_df)
X_test = test_df[train_features]
print(X_test.shape)

In [22]:
num_ensembles = 5
y_pred = 0.0
y_test_pred = 0.0
for i in tqdm(range(num_ensembles)):
    # TODO(you): Use CV, tune hyperparameters
    model = CatBoostRegressor(
        iterations=200, learning_rate=0.03,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model.fit(
        X_train_q1_q3, y_train_q1_q3,
        cat_features=cat_feature_inds)
    y_pred += model.predict(X_train_q4)
    y_test_pred += model.predict(X_test)
y_pred /= num_ensembles
y_test_pred /= num_ensembles

100%|██████████| 5/5 [17:29<00:00, 209.99s/it]


In [23]:
submission = pd.DataFrame({
    'ParcelId': test_df['parcelid'],
})
# https://www.kaggle.com/c/zillow-prize-1/discussion/33899, Oct,Nov,Dec
test_dates = {
    '201610': pd.Timestamp('2016-09-30'),
    '201611': pd.Timestamp('2016-10-31'),
    '201612': pd.Timestamp('2016-11-30'),
    '201710': pd.Timestamp('2017-09-30'),
    '201711': pd.Timestamp('2017-10-31'),
    '201712': pd.Timestamp('2017-11-30')
}
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    # TODO(you): predict for every `test_date`
    submission[label] = y_test_pred

submission_major = 3
submission.to_csv(
    'submission_%03d.csv' % (submission_major),
    float_format='%.4f',
    index=False)
print("Done! Good luck with submission #%d :)" % submission_major)

Predicting for: 201610 ... 
Predicting for: 201611 ... 
Predicting for: 201612 ... 
Predicting for: 201710 ... 
Predicting for: 201711 ... 
Predicting for: 201712 ... 
Done! Good luck with submission #3 :)


In [ ]:
np.mean(abs(y_pred-y_train_q4))

# No valTest Only

In [32]:
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(X_train[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)
        
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

Cat features are: ['airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'yearbuilt']


In [35]:
X_train.drop(['transaction_month', 'transaction_day', 'transaction_quarter'],axis=1, inplace=True)

/home/peter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    # TODO(you): Use CV, tune hyperparameters
    model = CatBoostRegressor(
        iterations=200, learning_rate=0.03,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model.fit(
        X_train, y_train,
        cat_features=cat_feature_inds)
    y_pred += model.predict(X_train)
y_pred /= num_ensembles

100%|██████████| 5/5 [05:53<00:00, 70.73s/it]


In [38]:
np.mean(abs(y_pred-y_train))

0.0660874724752986

In [15]:
np.mean(abs(y_pred-y_train))

0.06592959007432846

In [14]:
submission = pd.DataFrame({
    'ParcelId': test_df['parcelid'],
})
# https://www.kaggle.com/c/zillow-prize-1/discussion/33899, Oct,Nov,Dec
test_dates = {
    '201610': pd.Timestamp('2016-09-30'),
    '201611': pd.Timestamp('2016-10-31'),
    '201612': pd.Timestamp('2016-11-30'),
    '201710': pd.Timestamp('2017-09-30'),
    '201711': pd.Timestamp('2017-10-31'),
    '201712': pd.Timestamp('2017-11-30')
}
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    # TODO(you): predict for every `test_date`
    submission[label] = y_pred

submission_major = 1
submission.to_csv(
    'submission_%03d.csv' % (submission_major),
    float_format='%.4f',
    index=False)
print("Done! Good luck with submission #%d :)" % submission_major)

Predicting for: 201610 ... 
Predicting for: 201611 ... 
Predicting for: 201612 ... 
Predicting for: 201710 ... 
Predicting for: 201711 ... 
Predicting for: 201712 ... 
Done! Good luck with submission #1 :)
